In [1]:
import os
import numpy as np
import pandas as pd
import nltk
import keras
import six
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,AveragePooling1D,Flatten,concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import *

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package punkt to /home/roberto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/roberto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roberto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/roberto/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
#Abro el archivo en el que se encuentra el dataset de los problemas
with open('singleop.json', 'r') as f:
    datastore = json.load(f)

In [4]:
# Archivo que contiene un listado de nombres
nombres = pd.read_csv('nombres-2015.csv')
names = pd.read_csv('yob2019.txt', header=None)    

In [5]:
dataset2 = pd.read_csv('problemas_adicionales2.csv')

In [6]:
preguntas = []
respuestas = []
ecuaciones = []
alineacion = []

for item in datastore:
    preguntas.append(item['sQuestion'])
    respuestas.append(item['lSolutions'])
    ecuaciones.append(item['lEquations'])
    alineacion.append(item['lAlignments'])

In [7]:
# Necesito convertir el dataset en un problema de clasificacion para que la red neuronal pueda identificar
# si estoy tratando de resolver un problema de sumas, restas, multiplicaciones o divisiones.
# Esto va a crear una lista con el tipo de operacion y que va a ser el resultado a inferir.
operaciones = []
sumas =0
restas =0
multiplicaciones =0
divisiones = 0
otras = 0
#Clasifico las operaciones en 0 para sumas, 1 para restas, 2 para multiplicaciones, 3 para divisiones y 4 sino lo encuentro.
for operacion in ecuaciones:
    if (operacion[0].find('+')>=0):
        operaciones.append(0)
        sumas = sumas + 1
    elif (operacion[0].find('-') >= 0 ):
        operaciones.append(1)
        restas = restas + 1
    elif(operacion[0].find('*') >=0):
        operaciones.append(2)
        multiplicaciones = multiplicaciones + 1
    elif(operacion[0].find('/')):
        operaciones.append(3)
        divisiones = divisiones + 1
    else:
        operaciones.append(4)
        otras = otras + 1

print('Tengo ', sumas, ' sumas ', restas, ' restas, ', multiplicaciones, ' multiplicaciones, ', divisiones, ' divisiones y otras operaciones ', otras)

Tengo  159  sumas  162  restas,  117  multiplicaciones,  124  divisiones y otras operaciones  0


In [8]:
preguntas2 = dataset2['Preguntas'].tolist()
respuestas2 = dataset2['respuestas'].tolist()

In [9]:
preguntas3 = preguntas + preguntas2
respuestas3 = operaciones + respuestas2

In [10]:
#El listado de nombres lo voy a truncar a los 15K primeros, dado que el resto son nombres muy residuales.
nombres_ = nombres['nombre'][:15000]

In [11]:
names_= names[0]

In [12]:
nombres_ = nombres_.append(names_)

In [13]:
st_words = list(stop_words)

In [14]:
nom = nombres['nombre'].values.tolist()

In [15]:
nomb =  nom + st_words

In [16]:
nomb[-1]

'sea'

In [17]:
# El vector preguntas_sin, consiste en las preguntas a las que voy a eliminar todos los nombres propios que no
# anaden ningun valor al conjunto de preguntas. No quiero que esos nombres se procesen y por tanto los elimino.
def eliminar_palabras(dataset, stopw):
    preguntas_sin = []
    for palabras in dataset:
        frases = [word for word in palabras.split(' ') if word not in stopw]
        frases = " ".join(frases)
        preguntas_sin.append(frases)
    return preguntas_sin

In [18]:
preguntas_sin = eliminar_palabras(preguntas3, nomb)

In [19]:
from sklearn.utils import shuffle

preguntas3, respuestas3 = shuffle(preguntas_sin,respuestas3)

In [55]:
frases = []
tokenizer = RegexpTokenizer(r'\w+')

for i,pregunta in enumerate(preguntas_sin):
    palabras = tokenizer.tokenize(pregunta)
    preguntas_w = []
    for palabra in palabras:
        preguntas_w.append(palabra)
    frases.append(preguntas_w)

In [21]:
palabras_unicas = set(preguntas_w)

In [65]:
len(palabras_unicas)

2305

In [66]:
p_unicas = list(palabras_unicas)

#Voy a convertir los indices a escala logaritmica para evitar que puedan reventar los pesos en la red neuronal
vocabulario = {p:i*0.0001 for i, p in enumerate(p_unicas)}

In [67]:
tamanoMedio = 0
tamanoTotal = 0

for pregunta in frases:
    if(len(pregunta) > tamanoTotal):
        tamanoTotal = len(pregunta)
    tamanoMedio = tamanoMedio + len(pregunta)

print("El tamano max es ", tamanoTotal, " y la media de la longitud de las frases es de", tamanoMedio/len(frases))

El tamano max es  31  y la media de la longitud de las frases es de 12.803901437371664


In [87]:
# Voy a meter en este vector todas mis preguntas y todas las palabras.
# Para esta prueba, vamos a poner en la posición de la frase, el número de la palabra que estamos procesando.
# El objetivo es procesar las palabras teniendo en cuenta el orden secuencial de la frase.
bag = np.zeros([len(frases), tamanoTotal, 1])

In [88]:
for i, pregunta in enumerate(frases):
    for j, palabras in enumerate(pregunta):
        bag[i, tamanoTotal - len(pregunta) + j, 0] = vocabulario[palabras]

In [89]:
bag[0]

array([[0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.    ],
       [0.1292],
       [0.0123],
       [0.2296],
       [0.0049],
       [0.1921],
       [0.0834],
       [0.0123],
       [0.2296],
       [0.2054],
       [0.1958],
       [0.2296],
       [0.1921]])

In [90]:
training_X = np.asarray(bag[:800])
test_X = np.asarray(bag[800:])
training_y = np.asarray(respuestas3[:800])
test_y = np.asarray(respuestas3[800:])


In [91]:
# Creamos un modelo donde el primer argumento de la capa embedding son las palabras totales que voy a procesar
# vectorizadas en un indice.
# El segundo argumento, es el tamano del vector embedding, que he fijado en 16.
# El tercer argumento, es el tamano o longitud maxima, que he definido para las preguntas. Numero total de palabras
# por pregunta.
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(36, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [98]:
training_y

array([3, 1, 0, 0, 0, 1, 1, 1, 3, 2, 2, 2, 0, 1, 2, 1, 2, 3, 2, 0, 3, 0,
       0, 1, 2, 2, 3, 1, 1, 1, 2, 1, 3, 0, 2, 0, 2, 1, 2, 1, 3, 1, 3, 3,
       0, 1, 0, 0, 0, 2, 2, 0, 2, 0, 0, 1, 1, 2, 1, 2, 1, 3, 1, 0, 0, 1,
       1, 3, 2, 3, 2, 1, 3, 0, 2, 3, 1, 0, 0, 3, 3, 0, 1, 3, 1, 1, 2, 1,
       2, 2, 1, 3, 0, 0, 1, 1, 0, 1, 2, 3, 0, 1, 0, 1, 3, 2, 1, 1, 0, 0,
       1, 3, 2, 1, 2, 3, 2, 0, 0, 3, 1, 1, 2, 1, 2, 2, 0, 3, 0, 1, 3, 0,
       2, 3, 1, 3, 1, 1, 2, 2, 1, 0, 3, 2, 1, 1, 1, 3, 2, 0, 2, 2, 2, 2,
       2, 2, 1, 2, 2, 2, 3, 2, 3, 0, 1, 1, 1, 3, 3, 2, 2, 3, 1, 3, 0, 3,
       1, 1, 0, 2, 3, 0, 3, 2, 2, 0, 3, 2, 1, 0, 0, 3, 1, 1, 1, 3, 3, 1,
       0, 2, 1, 1, 1, 1, 1, 1, 3, 1, 1, 0, 3, 2, 1, 1, 2, 1, 1, 1, 0, 1,
       0, 2, 0, 0, 2, 1, 0, 1, 0, 0, 0, 3, 1, 1, 1, 1, 0, 2, 2, 1, 0, 2,
       1, 0, 3, 1, 2, 3, 2, 1, 1, 0, 0, 1, 2, 0, 1, 1, 2, 0, 1, 0, 3, 1,
       0, 0, 1, 2, 1, 2, 3, 2, 1, 1, 1, 3, 1, 0, 0, 2, 1, 1, 2, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 2, 3, 3, 1, 2, 1, 0, 1,

In [92]:
history = model.fit(training_X, training_y, epochs = 300, validation_data=(test_X, test_y), batch_size = 128)

Epoch 1/300
7/7 [==============================] - 0s 17ms/step - loss: 1.3834 - accuracy: 0.2825 - val_loss: 1.3788 - val_accuracy: 0.2529
Epoch 2/300
7/7 [==============================] - 0s 4ms/step - loss: 1.3645 - accuracy: 0.3500 - val_loss: 1.3711 - val_accuracy: 0.3448
Epoch 3/300
7/7 [==============================] - 0s 5ms/step - loss: 1.3514 - accuracy: 0.3812 - val_loss: 1.3672 - val_accuracy: 0.3448
Epoch 4/300
7/7 [==============================] - 0s 5ms/step - loss: 1.3383 - accuracy: 0.3787 - val_loss: 1.3682 - val_accuracy: 0.3448
Epoch 5/300
7/7 [==============================] - 0s 6ms/step - loss: 1.3341 - accuracy: 0.3787 - val_loss: 1.3745 - val_accuracy: 0.3448
Epoch 6/300
7/7 [==============================] - 0s 5ms/step - loss: 1.3343 - accuracy: 0.3787 - val_loss: 1.3808 - val_accuracy: 0.3448
Epoch 7/300
7/7 [==============================] - 0s 5ms/step - loss: 1.3329 - accuracy: 0.3787 - val_loss: 1.3807 - val_accuracy: 0.3448
Epoch 8/300
7/7 [=========

7/7 [==============================] - 0s 5ms/step - loss: 1.2836 - accuracy: 0.4038 - val_loss: 1.4282 - val_accuracy: 0.3276
Epoch 60/300
7/7 [==============================] - 0s 4ms/step - loss: 1.2805 - accuracy: 0.4075 - val_loss: 1.4263 - val_accuracy: 0.3276
Epoch 61/300
7/7 [==============================] - 0s 4ms/step - loss: 1.2808 - accuracy: 0.4087 - val_loss: 1.4294 - val_accuracy: 0.3161
Epoch 62/300
7/7 [==============================] - 0s 4ms/step - loss: 1.2792 - accuracy: 0.4038 - val_loss: 1.4339 - val_accuracy: 0.3103
Epoch 63/300
7/7 [==============================] - 0s 4ms/step - loss: 1.2762 - accuracy: 0.4125 - val_loss: 1.4339 - val_accuracy: 0.3161
Epoch 64/300
7/7 [==============================] - 0s 5ms/step - loss: 1.2847 - accuracy: 0.4087 - val_loss: 1.4365 - val_accuracy: 0.2989
Epoch 65/300
7/7 [==============================] - 0s 4ms/step - loss: 1.2738 - accuracy: 0.4137 - val_loss: 1.4314 - val_accuracy: 0.3161
Epoch 66/300
7/7 [===============

7/7 [==============================] - 0s 4ms/step - loss: 1.2307 - accuracy: 0.4412 - val_loss: 1.5017 - val_accuracy: 0.2931
Epoch 118/300
7/7 [==============================] - 0s 4ms/step - loss: 1.2292 - accuracy: 0.4275 - val_loss: 1.5065 - val_accuracy: 0.3046
Epoch 119/300
7/7 [==============================] - 0s 4ms/step - loss: 1.2233 - accuracy: 0.4550 - val_loss: 1.5020 - val_accuracy: 0.2989
Epoch 120/300
7/7 [==============================] - 0s 5ms/step - loss: 1.2179 - accuracy: 0.4375 - val_loss: 1.5063 - val_accuracy: 0.2874
Epoch 121/300
7/7 [==============================] - 0s 5ms/step - loss: 1.2136 - accuracy: 0.4400 - val_loss: 1.5059 - val_accuracy: 0.2989
Epoch 122/300
7/7 [==============================] - 0s 5ms/step - loss: 1.2081 - accuracy: 0.4563 - val_loss: 1.5053 - val_accuracy: 0.2874
Epoch 123/300
7/7 [==============================] - 0s 4ms/step - loss: 1.2154 - accuracy: 0.4288 - val_loss: 1.5005 - val_accuracy: 0.2874
Epoch 124/300
7/7 [========

7/7 [==============================] - 0s 5ms/step - loss: 1.1625 - accuracy: 0.4675 - val_loss: 1.5703 - val_accuracy: 0.2816
Epoch 176/300
7/7 [==============================] - 0s 6ms/step - loss: 1.1726 - accuracy: 0.4725 - val_loss: 1.5784 - val_accuracy: 0.2874
Epoch 177/300
7/7 [==============================] - 0s 5ms/step - loss: 1.1691 - accuracy: 0.4638 - val_loss: 1.5894 - val_accuracy: 0.3046
Epoch 178/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1650 - accuracy: 0.4650 - val_loss: 1.5727 - val_accuracy: 0.2816
Epoch 179/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1550 - accuracy: 0.4875 - val_loss: 1.5730 - val_accuracy: 0.2816
Epoch 180/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1582 - accuracy: 0.4787 - val_loss: 1.5862 - val_accuracy: 0.3046
Epoch 181/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1550 - accuracy: 0.4975 - val_loss: 1.5846 - val_accuracy: 0.2816
Epoch 182/300
7/7 [========

7/7 [==============================] - 0s 4ms/step - loss: 1.1264 - accuracy: 0.4888 - val_loss: 1.6515 - val_accuracy: 0.2874
Epoch 234/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1120 - accuracy: 0.5200 - val_loss: 1.6469 - val_accuracy: 0.2816
Epoch 235/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1297 - accuracy: 0.5088 - val_loss: 1.6573 - val_accuracy: 0.2816
Epoch 236/300
7/7 [==============================] - 0s 5ms/step - loss: 1.1102 - accuracy: 0.5200 - val_loss: 1.6494 - val_accuracy: 0.2874
Epoch 237/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1129 - accuracy: 0.5250 - val_loss: 1.6611 - val_accuracy: 0.2816
Epoch 238/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1287 - accuracy: 0.4837 - val_loss: 1.6593 - val_accuracy: 0.2989
Epoch 239/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1318 - accuracy: 0.4762 - val_loss: 1.6535 - val_accuracy: 0.2931
Epoch 240/300
7/7 [========

7/7 [==============================] - 0s 5ms/step - loss: 1.1327 - accuracy: 0.4913 - val_loss: 1.7070 - val_accuracy: 0.2989
Epoch 292/300
7/7 [==============================] - 0s 5ms/step - loss: 1.1220 - accuracy: 0.4938 - val_loss: 1.6889 - val_accuracy: 0.2989
Epoch 293/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1147 - accuracy: 0.5213 - val_loss: 1.6890 - val_accuracy: 0.2989
Epoch 294/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1003 - accuracy: 0.5125 - val_loss: 1.7201 - val_accuracy: 0.2931
Epoch 295/300
7/7 [==============================] - 0s 4ms/step - loss: 1.1067 - accuracy: 0.5038 - val_loss: 1.7115 - val_accuracy: 0.3046
Epoch 296/300
7/7 [==============================] - 0s 3ms/step - loss: 1.1023 - accuracy: 0.5250 - val_loss: 1.7077 - val_accuracy: 0.2931
Epoch 297/300
7/7 [==============================] - 0s 4ms/step - loss: 1.0954 - accuracy: 0.5150 - val_loss: 1.7105 - val_accuracy: 0.2931
Epoch 298/300
7/7 [========

In [101]:
model2 = keras.Sequential([
    keras.layers.Bidirectional(keras.layers.LSTM(128, dropout = 0.2, input_shape=[bag.shape[0], bag.shape[1], 1])),
    keras.layers.Dense(4, activation='softmax')
])
opt = keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
)

#opt = keras.optimizers.SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
#opt = keras.optimizers.Adam(learning_rate = 0.0001)
model2.compile(loss='sparse_categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

In [102]:
training_X.shape

(800, 31, 1)

In [103]:
model2.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [ ]:
history2 = model2.fit(training_X, training_y, epochs = 240, batch_size= 128, validation_data=(test_X, test_y))

Epoch 1/240
7/7 [==============================] - 1s 164ms/step - loss: 1.3709 - accuracy: 0.3438 - val_loss: 1.3646 - val_accuracy: 0.3448
Epoch 2/240
7/7 [==============================] - 1s 86ms/step - loss: 1.3428 - accuracy: 0.3787 - val_loss: 1.3655 - val_accuracy: 0.3448
Epoch 3/240
7/7 [==============================] - 1s 90ms/step - loss: 1.3345 - accuracy: 0.3787 - val_loss: 1.5903 - val_accuracy: 0.3448
Epoch 4/240
7/7 [==============================] - 1s 90ms/step - loss: 1.3734 - accuracy: 0.3787 - val_loss: 1.3657 - val_accuracy: 0.3448
Epoch 5/240
7/7 [==============================] - 1s 85ms/step - loss: 1.3355 - accuracy: 0.3787 - val_loss: 1.3821 - val_accuracy: 0.3448
Epoch 6/240
7/7 [==============================] - 1s 84ms/step - loss: 1.3365 - accuracy: 0.3787 - val_loss: 1.3668 - val_accuracy: 0.3448
Epoch 7/240
7/7 [==============================] - 1s 82ms/step - loss: 1.3354 - accuracy: 0.3787 - val_loss: 1.3645 - val_accuracy: 0.3448
Epoch 8/240
7/7 [==

7/7 [==============================] - 1s 101ms/step - loss: 1.3350 - accuracy: 0.3787 - val_loss: 1.3716 - val_accuracy: 0.3448
Epoch 60/240
7/7 [==============================] - 1s 95ms/step - loss: 1.3364 - accuracy: 0.3787 - val_loss: 1.3735 - val_accuracy: 0.3448
Epoch 61/240
7/7 [==============================] - 1s 90ms/step - loss: 1.3344 - accuracy: 0.3787 - val_loss: 1.3691 - val_accuracy: 0.3448
Epoch 62/240
7/7 [==============================] - 1s 85ms/step - loss: 1.3347 - accuracy: 0.3787 - val_loss: 1.3828 - val_accuracy: 0.3448
Epoch 63/240
7/7 [==============================] - 1s 83ms/step - loss: 1.3359 - accuracy: 0.3787 - val_loss: 1.3752 - val_accuracy: 0.3448
Epoch 64/240
7/7 [==============================] - 1s 84ms/step - loss: 1.3355 - accuracy: 0.3787 - val_loss: 1.3705 - val_accuracy: 0.3448
Epoch 65/240
7/7 [==============================] - 1s 122ms/step - loss: 1.3365 - accuracy: 0.3787 - val_loss: 1.3736 - val_accuracy: 0.3448
Epoch 66/240
7/7 [======

7/7 [==============================] - 1s 101ms/step - loss: 1.3406 - accuracy: 0.3787 - val_loss: 1.3701 - val_accuracy: 0.3448
Epoch 118/240
7/7 [==============================] - 1s 88ms/step - loss: 1.3318 - accuracy: 0.3787 - val_loss: 1.3740 - val_accuracy: 0.3448
Epoch 119/240
7/7 [==============================] - 1s 101ms/step - loss: 1.3328 - accuracy: 0.3800 - val_loss: 1.3773 - val_accuracy: 0.3448
Epoch 120/240
7/7 [==============================] - 1s 87ms/step - loss: 1.3319 - accuracy: 0.3787 - val_loss: 1.3701 - val_accuracy: 0.3448
Epoch 121/240
7/7 [==============================] - 1s 97ms/step - loss: 1.3316 - accuracy: 0.3787 - val_loss: 1.3720 - val_accuracy: 0.3448
Epoch 122/240
7/7 [==============================] - 1s 82ms/step - loss: 1.3309 - accuracy: 0.3787 - val_loss: 1.3714 - val_accuracy: 0.3448
Epoch 123/240
7/7 [==============================] - 1s 85ms/step - loss: 1.3279 - accuracy: 0.3812 - val_loss: 1.3739 - val_accuracy: 0.3448
Epoch 124/240
7/7 

Epoch 175/240
7/7 [==============================] - 1s 107ms/step - loss: 1.3263 - accuracy: 0.3725 - val_loss: 1.3852 - val_accuracy: 0.3448
Epoch 176/240
7/7 [==============================] - 1s 105ms/step - loss: 1.3226 - accuracy: 0.3750 - val_loss: 1.4082 - val_accuracy: 0.3391
Epoch 177/240
7/7 [==============================] - 1s 166ms/step - loss: 1.3492 - accuracy: 0.3762 - val_loss: 1.3858 - val_accuracy: 0.3448
Epoch 178/240
7/7 [==============================] - 1s 123ms/step - loss: 1.3267 - accuracy: 0.3738 - val_loss: 1.3891 - val_accuracy: 0.3448
Epoch 179/240
7/7 [==============================] - 1s 132ms/step - loss: 1.3276 - accuracy: 0.3825 - val_loss: 1.3832 - val_accuracy: 0.3448
Epoch 180/240
7/7 [==============================] - 1s 131ms/step - loss: 1.3220 - accuracy: 0.3775 - val_loss: 1.3884 - val_accuracy: 0.3448
Epoch 181/240
7/7 [==============================] - 1s 108ms/step - loss: 1.3279 - accuracy: 0.3750 - val_loss: 1.3936 - val_accuracy: 0.3448